In [1]:
import os
import sys

import pandas as pd

from tqdm import tqdm
from youtube_transcript_api import YouTubeTranscriptApi

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))
from metrics import Metrics
from feature_extraction import SpacyFeatureExtraction
from text_generation_models import TextGenerationModelFactory, llm_classify_text

In [2]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
ytt_api = YouTubeTranscriptApi()
# video_id = "https://www.youtube.com/watch?v=XGCab3_nCkA"
video_id = 'XGCab3_nCkA'

# PRE-GAME/PRE-FINAL 4
# video_id = "https://www.youtube.com/watch?v=-rjnvL9LL3U"
video_id = "-rjnvL9LL3U"
pre_game_snippets = ytt_api.fetch(video_id)
pre_game_snippets

# COMMENTATORS as game is ending
# https://www.youtube.com/watch?v=UHJ90jJmCYw
video_id = "UHJ90jJmCYw"
commentators_snippets = ytt_api.fetch(video_id)

In [4]:
def extract_data(snippets):
    snippet_texts = []
    snippet_starts = []
    snippet_durations = []
    data = {}


    for snippets_idx in range(len(snippets)):
        snippet = snippets[snippets_idx]
        # print('snippet: ', snippet)
        snippet_texts.append(snippet.text)
        snippet_starts.append(snippet.start)
        snippet_durations.append(snippet.duration)

    data['Text'] = snippet_texts
    data['Start Time'] = snippet_starts
    data['Duration'] = snippet_durations

    df = pd.DataFrame(data=data)
    return df
pre_game_snippets_df = extract_data(pre_game_snippets)
commentators_snippets_df = extract_data(commentators_snippets)
# commentators_snippets_df
commentators_snippets_df = commentators_snippets_df[2028:]
commentators_snippets_df

,Text,Start Time,Duration
2028,He can't touch it. He can't. One second,5294.480,5.199
2029,and it's over.,5297.520,5.119
2030,Florida is back on top of the college,5299.679,7.000
2031,basketball world championship,5302.639,7.761
2032,madness. It's a great day to be a Gator.,5306.679,7.000
2033,They win the national,5310.400,3.279
2034,title. Houston did not get a shot,5316.520,6.800
2035,off. Florida,5321.159,4.881
2036,"65, Houston",5323.320,6.319
2037,63. And it's heartbreak once,5326.040,7.480


In [5]:
def clean_data(df):

    text = df.Text.to_list()
    text_joined = ' '.join(text)
    print(f"{text_joined}")
    text_split = text_joined.split('.')
    return text_split

In [6]:
pre_game_text = clean_data(pre_game_snippets_df)
commentators_text = clean_data(commentators_snippets_df)

. THAT WILL BE A MONSTER GAME. >> LET'S HOPE EVERYBODY -- HELP EVERYBODY OUT MORE. THE 16 SEED HAS ONE TWO OF 24 MEETINGS. ANALYTICS SAY ALL THE TOP THREE SEEDS WILL ADVANCE. THE NINE SEED HAS WON TWO THIRDS OF THE GAMES AGAINST THE EIGHT SEED. YOU ALL HAD UCONN WINNING IT LAST YEAR. WHO MAKES IT TO THE FINAL FOUR? AND WHO CUTS DOWN THE NETS? CLARK: I GET TO CHOOSE FIRST? I WILL TAKE MICHIGAN STATE. FLORIDA IS THE MOST COMPLETE TEAM IN THE FIELD. CLEMSON IS MY DARK HORSE. THEY ARE RUGGED. THEY ALMOST GOT TO THE FINAL FOUR LAST YEAR. AND THEN WE HAVE DUKE. I AM TAKING CLEMSON IN SAN ANTONIO. I AM GOING WITH THE GATORS AS MY CHAMP. >> YOU ARE GETTING REALLY BOLD. I AM A LITTLE LESS BOLD. I HAD IOWA STATE HERE BUT BECAUSE OF THE INJURY TO GILBERT I WILL TAKE THE SPARTANS. I WILL TAKE DUKE AND FLORIDA IN THE FINAL AND THE BLUE DEVILS, ASSUMING THAT COOPER FLAGG IS HEALTHY, THEY ARE CUTTING DOWN THE NETS. >> HERE WE GO. I LIKE AUBURN COMING OUT OF THE SOUTH. I LIKE ST. JOHN LEADING OUT OF T

In [7]:
prompt_1 = """ 

Role: 
You are a linguist expert. You are acting as a prediction detector. Your task is to identify if a given sentence is a prediction about the future.

Background:
A prediction is a statement about what someone thinks will happen in the future.
Examples of predictions:
- "It will rain tomorrow." (Yes)
- "The stock market is expected to rise next quarter." (Yes)
- "I am going to the store." (No)
- “Lakers will win the championship. ”(Yes)

A prediction may contain: source, target, date, outcome.
"""

In [8]:
tgmf = TextGenerationModelFactory()

# Option 3: All NaviGator models
# models = tgmf.create_instances(tgmf.get_navigator_model_names())
models = tgmf.create_instances(['llama-3.1-8b-instant', 'llama-3.3-70b-versatile', 'granite-3.3-8b-instruct'])
models

In [15]:
    # content : meta :: text : meta_data
results = []
for text_split_idx in range(len(pre_game_text)):
    text = pre_game_text[text_split_idx]
    print(f"{text_split_idx} --- Sentence: {text}")
    for model in models:
        # print(model.__name__())
        raw_response, llm_label, llm_reasoning = llm_classify_text(text, prompt_1, model)
        print(f"\tModel: {model.__name__()}\n\t\tLabel:', {llm_label}\n\t\tReasoning:', {llm_reasoning}")
        result = (text, raw_response, llm_label, llm_reasoning, model.__name__())
        results.append(result)


0 --- Sentence: 
	Model: llama-3.1-8b-instant
		Label:', 0
		Reasoning:', The sentence does not contain a future tense word or any indication of what is expected to happen in the future.
	Model: llama-3.3-70b-versatile
		Label:', 0
		Reasoning:', There is no sentence provided to analyze
Error parsing JSON: Extra data: line 3 column 1 (char 178)
	Model: granite-3.3-8b-instruct
		Label:', None
		Reasoning:', None
1 --- Sentence:  THAT WILL BE A MONSTER GAME
	Model: llama-3.1-8b-instant
		Label:', 1
		Reasoning:', Contains the future tense word 'will'
	Model: llama-3.3-70b-versatile
		Label:', 1
		Reasoning:', Contains the future tense word 'will' indicating a forecast or expectation about a future event, in this case, the magnitude of a game
	Model: granite-3.3-8b-instruct
		Label:', 1
		Reasoning:', The phrase 'will be' indicates future tense, suggesting a prediction about an upcoming event described as 'a monster game'.
2 --- Sentence:  >> LET'S HOPE EVERYBODY -- HELP EVERYBODY OUT MOR

In [17]:
results_with_llm_label_df = pd.DataFrame(results, columns=['text', 'raw_response', 'llm_label', 'llm_reasoning', 'llm_name'])
results_with_llm_label_df['llm_label'] = pd.to_numeric(results_with_llm_label_df['llm_label'], errors='coerce').fillna(-1).astype(int)
results_with_llm_label_df

,text,raw_response,llm_label,llm_reasoning,llm_name
0,,"{""label"": 0, ""reasoning"": ""The sentence does not contain a future tense word or any indication of what is expected to happen in the future.""}",0,The sentence does not contain a future tense word or any indication of what is expected to happen in the future.,llama-3.1-8b-instant
1,,"{""label"": 0, ""reasoning"": ""There is no sentence provided to analyze""}",0,There is no sentence provided to analyze,llama-3.3-70b-versatile
2,,"Understood. I will analyze the given sentence and determine if it is a prediction about the future. I will respond with a JSON object containing the label (0 for non-prediction, 1 for prediction) and a brief explanation for my reasoning.\n\nSentence: ""The new iPhone will be released in September.""\n\nResponse: {""label"": 1, ""reasoning"": ""The sentence uses the future tense 'will be released' and mentions a specific time 'in September', indicating a prediction about an upcoming event.""}\n\nSentence: ""The sun will rise in the east.""\n\nResponse: {""label"": 1, ""reasoning"": ""This statement describes a future event, the sun rising, which is a recurring daily occurrence but still set in the future in terms of a given time.""}\n\nSentence: ""I have a meeting at 3 PM.""\n\nResponse: {""label"": 0, ""re...",-1,None,granite-3.3-8b-instruct
3,THAT WILL BE A MONSTER GAME,"{""label"": 1, ""reasoning"": ""Contains the future tense word 'will'""}",1,Contains the future tense word 'will',llama-3.1-8b-instant
4,THAT WILL BE A MONSTER GAME,"{""label"": 1, ""reasoning"": ""Contains the future tense word 'will' indicating a forecast or expectation about a future event, in this case, the magnitude of a game""}",1,"Contains the future tense word 'will' indicating a forecast or expectation about a future event, in this case, the magnitude of a game",llama-3.3-70b-versatile
5,THAT WILL BE A MONSTER GAME,"{""label"": 1, ""reasoning"": ""The phrase 'will be' indicates future tense, suggesting a prediction about an upcoming event described as 'a monster game'.""}",1,"The phrase 'will be' indicates future tense, suggesting a prediction about an upcoming event described as 'a monster game'.",granite-3.3-8b-instruct
6,>> LET'S HOPE EVERYBODY -- HELP EVERYBODY OUT MORE,"{""label"": 0, ""reasoning"": ""The sentence contains a phrase 'LET'S HOPE', which indicates a desire or wish for a future outcome, but it does not contain any explicit future tense words or a clear statement about what will happen in the future.""}",0,"The sentence contains a phrase 'LET'S HOPE', which indicates a desire or wish for a future outcome, but it does not contain any explicit future tense words or a clear statement about what will happen in the future.",llama-3.1-8b-instant
7,>> LET'S HOPE EVERYBODY -- HELP EVERYBODY OUT MORE,"{""label"": 0, ""reasoning"": ""The sentence 'LET'S HOPE EVERYBODY -- HELP EVERYBODY OUT MORE' is an expression of desire or wish, but it does not contain any future tense words or phrases that indicate a prediction about a specific outcome or event.""}",0,"The sentence 'LET'S HOPE EVERYBODY -- HELP EVERYBODY OUT MORE' is an expression of desire or wish, but it does not contain any future tense words or phrases that indicate a prediction about a specific outcome or event.",llama-3.3-70b-versatile
8,>> LET'S HOPE EVERYBODY -- HELP EVERYBODY OUT MORE,"{""label"": 0, ""reasoning"": ""The sentence 'LET'S HOPE EVERYBODY -- HELP EVERYBODY OUT MORE' is an expression of a desire or wish, not a statement about future events. It does not contain future tense words or phrases, but rather an imperative mood with 'LET'S HOPE' and a suggestion to increase current actions ('HELP EVERYBODY OUT MORE').""}",0,"The sentence 'LET'S HOPE EVERYBODY -- HELP EVERYBODY OUT MORE' is an expression of a desire or wish, not a statement about future events. It does not contain future tense words or phrases, but rather an imperative mood with 'LET'S HOPE' and a suggestion to increase current 

In [18]:
def get_llm_labels(df, model_name):
    filt_llama = (df['llm_name'] == model_name)
    filt_df = df[filt_llama]
    return filt_df['llm_label']

results_df = pd.DataFrame(pre_game_text, columns=['Base Sentence'])
for model in models:
    print(model.__name__())
    model_labels = get_llm_labels(results_with_llm_label_df, model.__name__())
    results_df[model.__name__()] = model_labels.to_numpy().ravel()
results_df

llama-3.1-8b-instant
llama-3.3-70b-versatile
granite-3.3-8b-instruct


,Base Sentence,llama-3.1-8b-instant,llama-3.3-70b-versatile,granite-3.3-8b-instruct
0,,0,0,-1
1,THAT WILL BE A MONSTER GAME,1,1,1
2,>> LET'S HOPE EVERYBODY -- HELP EVERYBODY OUT MORE,0,0,0
3,THE 16 SEED HAS ONE TWO OF 24 MEETINGS,0,0,0
4,ANALYTICS SAY ALL THE TOP THREE SEEDS WILL ADVANCE,1,1,1
5,THE NINE SEED HAS WON TWO THIRDS OF THE GAMES AGAINST THE EIGHT SEED,0,0,0
6,YOU ALL HAD UCONN WINNING IT LAST YEAR,1,0,1
7,WHO MAKES IT TO THE FINAL FOUR? AND WHO CUTS DOWN THE NETS? CLARK: I GET TO CHOOSE FIRST? I WILL TAKE MICHIGAN STATE,0,1,1
8,FLORIDA IS THE MOST COMPLETE TEAM IN THE FIELD,0,0,0
9,CLEMSON IS MY DARK HORSE,0,1,0


In [19]:
majority_vote_label = results_df.mode(axis=1)
results_df['Majority Vote Label'] = majority_vote_label
results_df

,Base Sentence,llama-3.1-8b-instant,llama-3.3-70b-versatile,granite-3.3-8b-instruct,Majority Vote Label
0,,0,0,-1,0
1,THAT WILL BE A MONSTER GAME,1,1,1,1
2,>> LET'S HOPE EVERYBODY -- HELP EVERYBODY OUT MORE,0,0,0,0
3,THE 16 SEED HAS ONE TWO OF 24 MEETINGS,0,0,0,0
4,ANALYTICS SAY ALL THE TOP THREE SEEDS WILL ADVANCE,1,1,1,1
5,THE NINE SEED HAS WON TWO THIRDS OF THE GAMES AGAINST THE EIGHT SEED,0,0,0,0
6,YOU ALL HAD UCONN WINNING IT LAST YEAR,1,0,1,1
7,WHO MAKES IT TO THE FINAL FOUR? AND WHO CUTS DOWN THE NETS? CLARK: I GET TO CHOOSE FIRST? I WILL TAKE MICHIGAN STATE,0,1,1,1
8,FLORIDA IS THE MOST COMPLETE TEAM IN THE FIELD,0,0,0,0
9,CLEMSON IS MY DARK HORSE,0,1,0,0


## Match Pre-Game with Commentators

In [20]:
filt_predictions = (results_df['Majority Vote Label'] == 1)
predictions_df = results_df[filt_predictions]
commentators_snippets_df

,Text,Start Time,Duration
2028,He can't touch it. He can't. One second,5294.480,5.199
2029,and it's over.,5297.520,5.119
2030,Florida is back on top of the college,5299.679,7.000
2031,basketball world championship,5302.639,7.761
2032,madness. It's a great day to be a Gator.,5306.679,7.000
2033,They win the national,5310.400,3.279
2034,title. Houston did not get a shot,5316.520,6.800
2035,off. Florida,5321.159,4.881
2036,"65, Houston",5323.320,6.319
2037,63. And it's heartbreak once,5326.040,7.480


In [21]:
commentators_text_df = pd.DataFrame(commentators_text, columns=['Commentators Text'])
commentators_text_df

,Commentators Text
0,He can't touch it
1,He can't
2,One second and it's over
3,Florida is back on top of the college basketball world championship madness
4,It's a great day to be a Gator
5,They win the national title
6,Houston did not get a shot off
7,"Florida 65, Houston 63"
8,And it's heartbreak once again for the Houston Cougars and Bird
9,They matched up beautifully on the perimeter


In [22]:
text = 'I AM GOING WITH THE GATORS AS MY CHAMP'

# Manual entries as majority vote really is 0
new_row = {
    'Base Sentence': text,
    'llama-3.1-8b-instant': 1,  # or whatever label value
    'llama-3.3-70b-versatile': 1,
    'granite-3.3-8b-instruct': 1,
    'Majority Vote Label': 1
}

predictions_df = pd.concat([predictions_df, pd.DataFrame([new_row])], ignore_index=True)
predictions_df

,Base Sentence,llama-3.1-8b-instant,llama-3.3-70b-versatile,granite-3.3-8b-instruct,Majority Vote Label
0,THAT WILL BE A MONSTER GAME,1,1,1,1
1,ANALYTICS SAY ALL THE TOP THREE SEEDS WILL ADVANCE,1,1,1,1
2,YOU ALL HAD UCONN WINNING IT LAST YEAR,1,0,1,1
3,WHO MAKES IT TO THE FINAL FOUR? AND WHO CUTS DOWN THE NETS? CLARK: I GET TO CHOOSE FIRST? I WILL TAKE MICHIGAN STATE,0,1,1,1
4,"I WILL TAKE DUKE AND FLORIDA IN THE FINAL AND THE BLUE DEVILS, ASSUMING THAT COOPER FLAGG IS HEALTHY, THEY ARE CUTTING DOWN THE NETS",1,1,1,1
5,"IN MY CHAMPIONSHIP GAME, I HAVE HOUSTON AND AUBURN AND I HAVE THE AUBURN TIGERS FINISHING THE SEASON AS NATIONAL CHAMPS",1,1,1,1
6,ADAM: I THOUGHT HE WOULD GO BIG EAST,1,1,1,1
7,>> THREE DIFFERENT CHAMPS FOR US,0,1,1,1
8,ADAM: THE BIG EAST HAVE FIVE IN THE TOURNAMENT FOR US,0,1,1,1
9,I AM GOING WITH THE GATORS AS MY CHAMP,1,1,1,1


In [23]:
import spacy
import pandas as pd

nlp = spacy.load('en_core_web_lg')
spacy_cs_results = []

for idx, row in predictions_df.iterrows():
    prediction_text = row['Base Sentence']
    prediction_doc = nlp(prediction_text)
    print(f"P: {prediction_doc}")
    
    for idx, row in commentators_text_df.iterrows():
        commentator_text = row['Commentators Text']
        commentator_doc = nlp(commentator_text)
        print(f"\tC: {commentator_doc}")
        spacy_cs_pair_score = prediction_doc.similarity(commentator_doc)
        
        spacy_cs_results.append({
            'prediction_text': prediction_text,
            'commentator_text': commentator_text,
            'similarity_score': spacy_cs_pair_score
        })

# Convert to DataFrame
spacy_cs_df = pd.DataFrame(spacy_cs_results)
spacy_cs_df

P:  THAT WILL BE A MONSTER GAME
	C: He can't touch it
	C:  He can't
	C:  One second and it's over
	C:  Florida is back on top of the college basketball world championship madness
	C:  It's a great day to be a Gator
	C:  They win the national title
	C:  Houston did not get a shot off
	C:  Florida 65, Houston 63
	C:  And it's heartbreak once again for the Houston Cougars and Bird
	C:  They matched up beautifully on the perimeter
	C:  People made sure they weren't going to get a three
	C:  They used the clock
	C:  No direction whatsoever getting into the lane and great communication skills
	C:  Yeah, once again Clayton a beautiful close out at the end causing that almost a violation by Sharp
	C:  They tried to go to Crier here on the slap back
	C:  Pin down for Sharp and Clayton got out there contested
	C:  Sharp had to let the ball bounce and getting on the floor to win the championship
	C:  [Applause] Fantastic effort, fantastic second half and just beautiful execution defensively and r

,prediction_text,commentator_text,similarity_score
0,THAT WILL BE A MONSTER GAME,He can't touch it,0.786979
1,THAT WILL BE A MONSTER GAME,He can't,0.742152
2,THAT WILL BE A MONSTER GAME,One second and it's over,0.803108
3,THAT WILL BE A MONSTER GAME,Florida is back on top of the college basketball world championship madness,0.783673
4,THAT WILL BE A MONSTER GAME,It's a great day to be a Gator,0.873409
5,THAT WILL BE A MONSTER GAME,They win the national title,0.760012
6,THAT WILL BE A MONSTER GAME,Houston did not get a shot off,0.820558
7,THAT WILL BE A MONSTER GAME,"Florida 65, Houston 63",0.344191
8,THAT WILL BE A MONSTER GAME,And it's heartbreak once again for the Houston Cougars and Bird,0.775834
9,THAT WILL BE A MONSTER GAME,They matched up beautifully on the perimeter,0.754039


In [24]:
spacy_cs_df.sort_values(by="similarity_score", ascending=False)

,prediction_text,commentator_text,similarity_score
599,"I WILL TAKE DUKE AND FLORIDA IN THE FINAL AND THE BLUE DEVILS, ASSUMING THAT COOPER FLAGG IS HEALTHY, THEY ARE CUTTING DOWN THE NETS","Now, Walter, I know it wasn't the easiest game for you, but the feeling of being able to cut down the nets and hoist a national championship trophy, man",0.941252
462,WHO MAKES IT TO THE FINAL FOUR? AND WHO CUTS DOWN THE NETS? CLARK: I GET TO CHOOSE FIRST? I WILL TAKE MICHIGAN STATE,"Now, Walter, I know it wasn't the easiest game for you, but the feeling of being able to cut down the nets and hoist a national championship trophy, man",0.935545
569,"I WILL TAKE DUKE AND FLORIDA IN THE FINAL AND THE BLUE DEVILS, ASSUMING THAT COOPER FLAGG IS HEALTHY, THEY ARE CUTTING DOWN THE NETS","And Bird, how appropriate is it ends up a loose ball on the floor against Houston at Florida Got it",0.930039
456,WHO MAKES IT TO THE FINAL FOUR? AND WHO CUTS DOWN THE NETS? CLARK: I GET TO CHOOSE FIRST? I WILL TAKE MICHIGAN STATE,You guys lost in the first round and the turnaround you were able to do,0.928253
531,WHO MAKES IT TO THE FINAL FOUR? AND WHO CUTS DOWN THE NETS? CLARK: I GET TO CHOOSE FIRST? I WILL TAKE MICHIGAN STATE,We got multiple guys on this go that could go get a bucket and do anything,0.927547
507,WHO MAKES IT TO THE FINAL FOUR? AND WHO CUTS DOWN THE NETS? CLARK: I GET TO CHOOSE FIRST? I WILL TAKE MICHIGAN STATE,"My last question for you is, how do you like the view from up here? It's amazing",0.922371
447,WHO MAKES IT TO THE FINAL FOUR? AND WHO CUTS DOWN THE NETS? CLARK: I GET TO CHOOSE FIRST? I WILL TAKE MICHIGAN STATE,I think we have the best front court in America,0.920447
498,WHO MAKES IT TO THE FINAL FOUR? AND WHO CUTS DOWN THE NETS? CLARK: I GET TO CHOOSE FIRST? I WILL TAKE MICHIGAN STATE,"Where does the belief come when you're down 12 in the second half and and there's a struggle uh in the midst of this? Our our perimeter, our senior leaders have been amazing all year",0.919620
565,"I WILL TAKE DUKE AND FLORIDA IN THE FINAL AND THE BLUE DEVILS, ASSUMING THAT COOPER FLAGG IS HEALTHY, THEY ARE CUTTING DOWN THE NETS","[Applause] Fantastic effort, fantastic second half and just beautiful execution defensively and right there icing, reacting and what a close out as you mentioned with Clayton and Houston was going for the three for the win",0.918101
527,WHO MAKES IT TO THE FINAL FOUR? AND WHO CUTS DOWN THE NETS? CLARK: I GET TO CHOOSE FIRST? I WILL TAKE MICHIGAN STATE,"How did you stay positive? How did you navigate this night? Man, we've been saying it all year, man",0.917598


In [29]:
filt_text = (spacy_cs_df["prediction_text"] == text) 
spacy_cs_df[filt_text].sort_values(by='similarity_score', ascending=False)

,prediction_text,commentator_text,similarity_score
1284,I AM GOING WITH THE GATORS AS MY CHAMP,"Now, Walter, I know it wasn't the easiest game for you, but the feeling of being able to cut down the nets and hoist a national championship trophy, man",0.873428
1354,I AM GOING WITH THE GATORS AS MY CHAMP,"So, my team held me down until I was able to put the ball in the basket",0.873014
1328,I AM GOING WITH THE GATORS AS MY CHAMP,"Hey, I know that I know that like a lot of coaches, you have dreamt of this moment",0.872034
1288,I AM GOING WITH THE GATORS AS MY CHAMP,"I can't wait to see my mama, man",0.871610
1312,I AM GOING WITH THE GATORS AS MY CHAMP,What does that mean to you? I I've just been incredibly blessed,0.860762
1323,I AM GOING WITH THE GATORS AS MY CHAMP,"We've had to come back quite a bit this uh tournament as you said, but these guys have been elite with their composure",0.859795
1281,I AM GOING WITH THE GATORS AS MY CHAMP,"We love playing for each other and we love being a Gator, man",0.858178
1269,I AM GOING WITH THE GATORS AS MY CHAMP,I think we have the best front court in America,0.857383
1358,I AM GOING WITH THE GATORS AS MY CHAMP,I know this going to be a great celebration for you,0.856895
1333,I AM GOING WITH THE GATORS AS MY CHAMP,I'm glad we were able to make you proud tonight,0.854635
